In [9]:
import pandas as pd
import folium
from folium.features import GeoJsonTooltip
import requests

In [10]:
# Path to your CSV file
csv_file_path = '../data/data.csv'
variable_options = [
    'Total bridges',
    'Bridges, poor',
    'Bridges, fair',
    'Bridges, good',
    'Bridge area (square meters)',
    'Bridge area, poor (square meters)',
    'Bridge area, fair (square meters)',
    'Bridge area, good (square meters)'
]

df = pd.read_csv(csv_file_path)

In [11]:
name_mapping = {
    'Total bridges': 'Total Bridges',
    'Bridges, poor': 'Total Poor Bridges',
    'Bridges, fair': 'Total Fair Bridges',
    'Bridges, good': 'Total Good Bridges',
    'Bridge area (square meters)': 'Total Bridge Area (k m²)',
    'Bridge area, poor (square meters)': 'Total Poor Bridge Area (k m²)',
    'Bridge area, fair (square meters)': 'Total Fair Bridge Area (k m²)',
    'Bridge area, good (square meters)': 'Total Good Bridge Area (k m²)'
}

# Define color scheme for each category
color_scheme = {
    'Total': 'Blues',
    'Poor': 'Reds',
    'Fair': 'Oranges',
    'Good': 'Greens'
}

# Update the 'Variable' column in the DataFrame based on the mapping
df['Variable'] = df['Variable'].map(name_mapping)

# Update variable_options list using the new names from the mapping
variable_options = [name_mapping[var] for var in variable_options]

# Optionally, adjust values for area measurements
area_variables = [name for name in name_mapping.values() if 'Area' in name]
for var in area_variables:
    df.loc[df['Variable'] == var, 'Value'] /= 1000  # Divide by 1000 for area values

In [12]:
folium_states_url = "https://raw.githubusercontent.com/python-visualization/folium-example-data/main/us_states.json"
us_states = requests.get(folium_states_url).json()

# Remove HI, AK, DC from the Folium GeoJSON data
states_to_drop = ['HI','AK','DC']
us_states['features'] = [f for f in us_states['features'] if f['id'] not in states_to_drop]

In [13]:
# Create a folium Map object, centered on the US, *without* a tile layer yet
m = folium.Map(
    [43, -100],
    zoom_start=4,
    tiles=None
)
# # Add the TileLayer to m *separately*, with control=False so that users
# # are not able remove the layer by unchecking a checkbox
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(m)

In [14]:
# Loop through each variable option to create separate choropleth layers
for variable in variable_options:
    subset_df = df[df['Variable'] == variable]
    legend_label = variable.replace('_', ' ').title()
    if 'Poor' in variable:
        color = color_scheme['Poor']
    elif 'Fair' in variable:
        color = color_scheme['Fair']
    elif 'Good' in variable:
        color = color_scheme['Good']
    else:  # Default to 'Total' for any other cases, typically 'Total'
        color = color_scheme['Total']

    choro = folium.Choropleth(
        geo_data=us_states,
        name=variable,
        data=subset_df,
        columns=['State', 'Value'],
        key_on='feature.properties.name',
        fill_color=color,
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=legend_label,
        highlight=True,
        show=(variable == 'Total Bridges'),
        overlay=False
    ).add_to(m)

    # Add tooltips to each layer
    tooltip = GeoJsonTooltip(
        fields=['name'],
        aliases=['State:'],
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
        max_width=800,
    )
    choro.geojson.add_child(tooltip)

In [15]:
folium.LayerControl(collapsed=False).add_to(m)

In [16]:
m